<img src="https://github.com/mousastech/medallion/blob/fd1da67c7e3e3829e0ea84fc51c6c79a02e408da/imgs/Medallion.png?raw=true">

#Medallion Architecture
With Unity Catalog

<img src="https://github.com/mousastech/medallion/blob/92d8750f657288477d48ba7e07ac8c8340d49cf3/imgs/architecture.png?raw=true">

[Reference](https://www.databricks.com/glossary/medallion-architecture)

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType

In [0]:
# Point out the external location mapped before
# Unity Catalog manages all permissions 

catalog = "ecommerce"
schema = "gold"

#Loading Gold layer
Business-level Aggregates

In [0]:
%sql
-- Criando a camada Gold, gerando uma tabela juntando todos os dados
CREATE OR REPLACE TABLE ecommerce.gold.sales
USING DELTA PARTITIONED BY (estadoCliente) 
AS
SELECT
  CASE
    WHEN orders_bronze.order_status = 'shipped' THEN 'enviado'
    WHEN orders_bronze.order_status = 'canceled' THEN 'cancelado'
    WHEN orders_bronze.order_status = 'invoiced' THEN 'faturado'
    WHEN orders_bronze.order_status = 'created' THEN 'criado'
    WHEN orders_bronze.order_status = 'delivered' THEN 'entregue'
    WHEN orders_bronze.order_status = 'unavailable' THEN 'indisponível'
    WHEN orders_bronze.order_status = 'processing' THEN 'em processamento'
    WHEN orders_bronze.order_status = 'approved' THEN 'aprovado'
  END AS statusDoPedido,
  orders_bronze.order_purchase_timestamp AS horaCompraPedido,
  orders_bronze.order_approved_at AS horaPedidoAprovado,
  orders_bronze.order_estimated_delivery_date AS dataEstimadaEntrega,
  DATEDIFF(
    orders_bronze.order_estimated_delivery_date,
    orders_bronze.order_approved_at
  ) AS dataEntregaEmDias,
  order_reviews.review_score AS notaProduto,
  order_reviews.review_answer_timestamp AS dataComentarioSobreProduto,
  CASE
    WHEN order_payments.paymentType = 'credit_card' THEN 'cartao_de_credito'
    WHEN order_payments.paymentType = 'boleto' THEN 'boleto'
    WHEN order_payments.paymentType = 'not_defined' THEN 'não_definido'
    WHEN order_payments.paymentType = 'voucher' THEN 'voucher'
    WHEN order_payments.paymentType = 'debit_card' THEN 'cartao_de_debito'
  END AS meioDePagamento,
  order_payments.paymentInstallments AS parcelamento,
  order_payments.paymentValue AS valorPago,
  customers.customer_city AS cidadeCliente,
  customers.customer_state AS estadoCliente
FROM
  ecommerce.bbronze.orders_bronze orders_bronze
  LEFT JOIN ecommerce.bbronze.order_payments_bronze order_payments 
    ON orders_bronze.order_id = order_payments.order_id
  LEFT JOIN ecommerce.bbronze.order_reviews_bronze order_reviews 
    ON orders_bronze.order_id = order_reviews.order_id
  LEFT JOIN ecommerce.bbronze.customers_bronze customers 
    ON orders_bronze.customer_id = customers.customer_id

In [0]:
from pyspark.sql.functions import col, when, datediff

# Read the necessary tables into DataFrames
orders = spark.table("ecommerce.bbronze.orders_bronze")
order_payments = spark.table("ecommerce.bbronze.order_payments_bronze")
order_reviews = spark.table("ecommerce.bbronze.order_reviews_bronze")
customers = spark.table("ecommerce.bbronze.customers_bronze")

# Perform the necessary joins and transformations
sales = (
    orders
    .join(order_payments, orders.order_id == order_payments.orderId, "left")
    .join(order_reviews, orders.order_id == order_reviews.order_id, "left")
    .join(customers, orders.customer_id == customers.customer_id, "left")
    .select(
        when(col("orders_bronze.order_status") == 'shipped', 'enviado')
        .when(col("orders_bronze.order_status") == 'canceled', 'cancelado')
        .when(col("orders_bronze.order_status") == 'invoiced', 'faturado')
        .when(col("orders_bronze.order_status") == 'created', 'criado')
        .when(col("orders_bronze.order_status") == 'delivered', 'entregue')
        .when(col("orders_bronze.order_status") == 'unavailable', 'indisponível')
        .when(col("orders_bronze.order_status") == 'processing', 'em processamento')
        .when(col("orders_bronze.order_status") == 'approved', 'aprovado')
        .alias("statusDoPedido"),
        col("orders_bronze.order_purchase_timestamp").alias("horaCompraPedido"),
        col("orders_bronze.order_approved_at").alias("horaPedidoAprovado"),
        col("orders_bronze.order_estimated_delivery_date").alias("dataEstimadaEntrega"),
        datediff(col("orders_bronze.order_estimated_delivery_date"), col("orders_bronze.order_approved_at")).alias("dataEntregaEmDias"),
        col("order_reviews_bronze.review_score").alias("notaProduto"),
        col("order_reviews_bronze.review_answer_timestamp").alias("dataComentarioSobreProduto"),
        when(col("order_payments_bronze.paymentType") == 'credit_card', 'cartao_de_credito')
        .when(col("order_payments_bronze.paymentType") == 'boleto', 'boleto')
        .when(col("order_payments_bronze.paymentType") == 'not_defined', 'não_definido')
        .when(col("order_payments_bronze.paymentType") == 'voucher', 'voucher')
        .when(col("order_payments_bronze.paymentType") == 'debit_card', 'cartao_de_debito')
        .alias("meioDePagamento"),
        col("order_payments_bronze.paymentInstallments").alias("parcelamento"),
        col("order_payments_bronze.paymentValue").alias("valorPago")
    )
)

#display(sales)

# Write the DataFrame to a Delta table in the silver schema with schema merge enabled
table_name = "ecommerce.gold.sales"

sales.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(table_name)

In [0]:
%sql

SELECT
  estadoCliente AS estados,
  meioDePagamento AS `meio de pagamento`,
  count(*) AS `percentual`
FROM
  ecommerce.gold.sales
WHERE
  meioDePagamento IS NOT NULL
  AND YEAR(horaPedidoAprovado) IS NOT NULL
  AND statusDoPedido = "entregue"
GROUP BY
  estadoCliente,
  meioDePagamento

In [0]:
%sql
select * from ecommerce.gold.sales

# Views

In [0]:
%sql

SELECT
  --estadoCliente AS estados, 
  meioDePagamento AS `meio de pagamento`
  , COUNT(*) AS `percentual`
FROM
  ecommerce.gold.sales
WHERE
  meioDePagamento IS NOT NULL
  AND YEAR(horaPedidoAprovado) IS NOT NULL
  AND statusDoPedido = "entregue"
GROUP BY
  --estadoCliente, 
  meioDePagamento

Databricks visualization. Run in Databricks to view.

In [0]:
%sql 

SELECT
  T.estadoCliente AS estados,
  T.diasEntrega AS `média de dias para entrega de produto`
FROM
  (
    SELECT
      estadoCliente,
      ROUND(AVG(dataEntregaEmDias), 0) AS DiasEntrega
    FROM
      tutorial.original.sales
    WHERE
      meioDePagamento IS NOT NULL
      AND YEAR(horaPedidoAprovado) IS NOT NULL
      AND statusDoPedido <> "cancelado"
    GROUP BY
      estadoCliente
  ) AS T

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
    estadoCliente as `Estado`,
    Year(horaPedidoAprovado) as `Ano`,
    Count(*) as `Numero de Vendas`
FROM
    tutorial.original.sales
WHERE
    statusDoPedido = "entregue"
And
    Year(horaPedidoAprovado) = "2018"
GROUP BY
    Estado, Ano


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

SELECT
  ROUND(SUM(valorPago)/Count(*), 2) as `Ticket Médio`,
  Month(horaPedidoAprovado)
FROM
  tutorial.original.sales
WHERE
  statusDoPedido = "entregue" AND Year(horaPedidoAprovado) = "2017"
GROUP BY
  Month(horaPedidoAprovado)

Databricks visualization. Run in Databricks to view.